In [1]:
from flask import Flask, render_template , redirect , request , session,flash
import json
import csv
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from function_final import predict_responses
import random

In [2]:
answer = {'name': 'testing oooooooo', 'q1': 'yes', 'q2': 'no', 'q3': 'yes', 'q4': 'no', 'q7': 'yes', 'q8': 'no', 'q9': 'yes', 'q10': 'no', 'q12': 'yes', 'q13': 'no', 'q14': 'yes', 'q15': 'no', 'q16': 'yes', 'q17': 'no', 'q18': 'yes', 'q19': 'no', 'q20': 'yes', 'q21': 'no', 'q22': 'yes', 'q23': 'no', 'q24': 'yes', 'q27': 'no', 'q28': 'yes', 'q29': 'no', 'q30': 'yes', 'q31': 'no', 'q32': 'yes', 'q33': 'no', 'q34': 'yes', 'q35': 'no'}

In [41]:
def predict_responses_randon_qa(answer):
    df = pd.read_csv('answer.csv')
    # Drop the 'name' column as it's not used for similarity calculation
    df = df.drop('name', axis=1)
    # Convert categorical responses to numerical values
    df = df.replace({'yes': 1, 'no': 0})

    # getting the dictionary and removing the column name
    answer.pop('name', None)
    # getting the list of index that the answer is given
    answer_index = list(answer.keys())
    # getting the list of answered question and not answered question
    X = df[answer_index]
    y = df.drop(columns=answer_index)
    # converting answer to 1/0
    new_user_responses = {key : 1 if value == 'yes' else 0 for key,value in answer.items()}
    # converting the response to dataframe
    new_user_df = pd.DataFrame([new_user_responses])
    # Compute cosine similarity between the new user and existing users
    similarities = cosine_similarity(new_user_df, X)
    # Find the indices that would sort the array in descending order
    sorted_indices = np.argsort(similarities[0])[::-1]
    #sorting the values in desending order where max similarity is in front
    # Sort the similarities array based on the sorted indices
    sorted_similarities = similarities[0, sorted_indices]
    # taking top 5 users and getting the prediction value from top 5 most similar below
    top_5_list = sorted_indices[:5]
    top_5_lists_values = sorted_similarities[:5]
    # getting the top 5 values from the dataframe using index and making dataframe 
    op_df = pd.DataFrame()
    opDf = pd.DataFrame()
    # converting the top 5 list to dictionary for threshold calculation
    op_dict = {}
    threshold_values_index = []
    for i in range(5):
        op_dict[top_5_list[i]] = top_5_lists_values[i]
    print(op_dict)
    print('------'*30)
    threshold = 0.9
    for i,v in op_dict.items():
        if threshold < v:
            threshold_values_index.append(i) 

    if threshold_values_index:
        for i in threshold_values_index:
            valuesToAppend = df.loc[i,list(y.keys())].values
            opDf = pd.concat([opDf , pd.DataFrame(data = [valuesToAppend] ,columns = list(y.keys()) )] , ignore_index=True)
        print('opDf' , opDf)
        output = {}
        for col in list(y.keys()):
            val = list(opDf.loc[: ,col].mode())
            output.update({col : val})
        print(output, ' recomm')
        return output
    else:
        pass 

    for i in top_5_list:
        values_to_append = df.loc[i,list(y.keys())].values
        op_df = pd.concat([op_df , pd.DataFrame(data = [values_to_append] ,columns = list(y.keys()) )] , ignore_index=True)
    print(op_df , 'old ')
    output = {}
    for col in list(y.keys()):
        val = list(op_df.loc[: ,col].mode())
        output.update({col : val})
    return output

In [42]:

preditcted_answer = predict_responses_randon_qa(answer)
my_answer  = {'q5' : 0 , 'q6' : 0 , 'q11' :1 , 'q25' : 1 , 'q26' : 0}

print(preditcted_answer , '\n' , my_answer)

{9: 0.9999999999999998, 6: 0.6831300510639732, 5: 0.6210590034081188, 3: 0.6210590034081188, 0: 0.6197797868009121}
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
opDf    q5  q6  q11  q25  q26
0   1   0    1    1    1
{'q5': [1], 'q6': [0], 'q11': [1], 'q25': [1], 'q26': [1]}  recomm
{'q5': [1], 'q6': [0], 'q11': [1], 'q25': [1], 'q26': [1]} 
 {'q5': 0, 'q6': 0, 'q11': 1, 'q25': 1, 'q26': 0}


In [39]:
from sklearn.metrics import mean_squared_error
my_answer_values = list(my_answer.values())
predicted_answer_values = list(preditcted_answer.values())

mse = mean_squared_error(my_answer_values , predicted_answer_values)
mse

0.4

In [40]:
df = pd.read_csv('answer.csv')
df.head()

,name,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q26,q27,q28,q29,q30,q31,q32,q33,q34,q35
0,test 35qns,yes,yes,yes,yes,yes,yes,yes,yes,yes,...,no,no,no,no,no,no,no,no,no,no
1,test pred 35 qns,yes,yes,no,yes,yes,no,yes,yes,no,...,no,no,no,no,no,no,no,no,no,no
2,last test cont q35,no,no,no,no,no,no,no,no,no,...,no,no,no,no,no,yes,yes,yes,yes,yes
3,pred test 30qns,yes,no,yes,no,yes,no,yes,no,yes,...,yes,no,no,no,yes,no,yes,no,yes,no
4,ppppppppppppppppppppp,yes,no,yes,no,yes,no,yes,no,yes,...,no,yes,no,yes,no,no,yes,no,no,no


name    testing oooooooo
q1                   yes
q2                    no
q3                   yes
q4                    no
q5                   yes
q6                    no
q7                   yes
q8                    no
q9                   yes
q10                   no
q11                  yes
q12                  yes
q13                   no
q14                  yes
q15                   no
q16                  yes
q17                   no
q18                  yes
q19                   no
q20                  yes
q21                   no
q22                  yes
q23                   no
q24                  yes
q25                  yes
q26                  yes
q27                   no
q28                  yes
q29                   no
q30                  yes
q31                   no
q32                  yes
q33                   no
q34                  yes
q35                   no
Name: 9, dtype: object


In [6]:
a =1 
b=2
c=3
d=4

def summ(*a):
    ss = 0
    for i in a:
        ss+=i
    return ss

print(summ(a,b))
print(summ(a,b,c))
print(summ(a,b,c , d))

3
6
10
